<a href="https://colab.research.google.com/github/withlionbuddha/learning.ai/blob/ground/PseudoLabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

의사 레이블(Pseudo-label)

---
의사 레이블링은 일반적으로 레이블이 일부만 주어졌을 때, 나머지 데이터를 예측하여 성능을 개선하는데 사용됩니다



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

In [11]:
import os
import shutil
# MNIST 데이터셋이 저장된 경로
cache_dir = os.path.expanduser('~/.keras/datasets/mnist.npz')

# 캐시 파일 삭제
if os.path.exists(cache_dir):
    os.remove(cache_dir)
    print("Cached MNIST data deleted.")

Cached MNIST data deleted.


mnist.load_data()
   손으로 쓴 숫자(0-9) 이미지를 모아둔 데이터셋

xtrain, y_train, x_test, y_test
    0~255 사이의 정수 값(픽셀 값)으로 이루어진 배열


In [12]:
# MNIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


 이미지 데이터(28x28 픽셀)를 60,000개의 샘플로 쌓아올린 3D 배열



In [15]:
print(f"x_train shape: {x_train.shape}")
print(f"axis=0 (number of images): {x_train.shape[0]}")
print(f"axis=1 (height of each image): {x_train.shape[1]}")
print(f"axis=2 (width of each image): {x_train.shape[2]}")

x_train shape: (60000, 28, 28)
axis=0 (number of images): 60000
axis=1 (height of each image): 28
axis=2 (width of each image): 28


In [7]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [2]:




# 데이터 전처리
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 레이블이 있는 데이터를 일부만 사용하고 나머지는 의사 레이블로 처리
# 여기서는 예시로 y_train의 앞부분만 사용
num_labeled = 1000
x_labeled = x_train[:num_labeled]
y_labeled = y_train[:num_labeled]
x_unlabeled = x_train[num_labeled:]
y_unlabeled = y_train[num_labeled:]  # 원래 라벨은 실제 학습에 사용되지 않음

# 간단한 모델 구성
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 먼저 라벨이 있는 데이터로 학습
model.fit(x_labeled, y_labeled, epochs=5, batch_size=32, validation_split=0.2)

# 의사 레이블 생성: 레이블이 없는 데이터를 모델을 통해 예측
pseudo_labels = np.argmax(model.predict(x_unlabeled), axis=1)

# 라벨이 없는 데이터에 대해 예측된 의사 레이블로 다시 학습
x_combined = np.concatenate([x_labeled, x_unlabeled])
y_combined = np.concatenate([y_labeled, pseudo_labels])

model.fit(x_combined, y_combined, epochs=5, batch_size=32, validation_split=0.2)

# 테스트 데이터로 성능 확인
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.3915 - loss: 1.9721 - val_accuracy: 0.7850 - val_loss: 1.0300
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8425 - loss: 0.7671 - val_accuracy: 0.8300 - val_loss: 0.6708
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8965 - loss: 0.4253 - val_accuracy: 0.8500 - val_loss: 0.5719
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9302 - loss: 0.3105 - val_accuracy: 0.8650 - val_loss: 0.5236
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9439 - loss: 0.2461 - val_accuracy: 0.8550 - val_loss: 0.5277
1844/1844 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9542 - loss: 0.1605 - val_accuracy: 0.9606 - val_loss: 0.0986
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9663 - loss: 0.0915 - val_accuracy: 0.9671 - val_loss: 0.0796
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9727 - loss: 0.